# Output tags in BIO format for NER analysis
### Make function that takes a vocab list and a set of MWOs and returns MWOs in bio format
#### TODO: BIO or IOB?
#### TODO: Handle SI, PI, longer ngrams

#### https://pythonprogramming.net/using-bio-tags-create-named-entity-lists/

In [1]:
import pandas as pd
import json
from nestor import keyword as kex

/Users/amc8/miniconda3/envs/bio_output/lib/python3.8/site-packages/poetry_dynamic_versioning/__init__.py:276: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  module = _state.original_import_func(name, globals, locals, fromlist, level)


In [2]:
# Get raw MWOs
df = (pd.read_csv('/Users/amc8/Documents/datasets/MWOs.csv')
      .rename(columns={'BscStartDate': 'StartDate'})
      )

# Change date column to DateTime objects
df.StartDate = pd.to_datetime(df['StartDate'])
df.head(5)

,StartDate,Asset,OriginalShorttext,PMType,Cost,RunningTime,MajorSystem,Part,Action,Variant,FM,Location,Comments,FuncLocation,SuspSugg,Rule
0,2004-03-17,A,JUMP START,PM01,0.00,57.0,Engine,Starting System,Minor Maint,15,NaN,NaN,NaN,CHARGING SYSTEM,0,Rule_1_3_50_245_248_353
1,2005-10-02,A,JUMP START,PM01,425.88,20.0,Engine,Starting System,Minor Maint,15,NaN,NaN,NaN,STARTING SYSTEMS,0,Rule_1_3_50_245_248_353
2,2005-11-07,A,JUMP START,PM01,435.97,20.0,Engine,Starting System,Minor Maint,15,NaN,NaN,NaN,STARTING SYSTEMS,0,Rule_1_3_50_245_248_353
3,2005-11-15,A,JUMP START,PM01,0.00,8.0,Engine,Starting System,Minor Maint,15,NaN,NaN,NaN,STARTING SYSTEMS,0,Rule_1_3_50_245_248_353
4,2005-11-17,A,JUMP START,PM01,0.00,2.0,Engine,Starting System,Minor Maint,15,NaN,NaN,NaN,BATTERIES,0,Rule_1_3_50_245_248_353


In [3]:
# Get tagged MWOs
tags = pd.read_csv('/Users/amc8/Documents/datasets/READABLE_TAGS.csv')
tags.head(5)

,Unnamed: 0,BscStartDate,Asset,OriginalShorttext,PMType,Cost,I,P,S,U,X,P I,S I
0,0,7/1/04,A,BUCKET WON'T OPEN,PM01,183.05,bucket,NaN,NaN,"won, open",NaN,NaN,NaN
1,1,3/20/05,A,L/H BUCKET CYL LEAKING.,PM01,407.40,"bucket, cylinder, bucket_cylinder",leak,NaN,NaN,NaN,cylinder leak,NaN
2,2,5/5/06,A,SWAP BUCKET,PM01,0.00,bucket,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7/11/06,A,FIT BUCKET TOOTH,PM01,0.00,"bucket, tooth, bucket_tooth",NaN,NaN,NaN,NaN,NaN,NaN
4,4,11/10/06,A,REFIT BUCKET TOOTH,PM01,1157.27,"bucket, tooth, bucket_tooth",NaN,NaN,NaN,NaN,NaN,refit bucket


In [4]:
# Get vocab file (unigrams)
vocab = pd.read_csv('/Users/amc8/Documents/datasets/1g_original.csv')
vocab.head(5)

,tokens,NE,alias,notes,score
0,replace,S,replace,NaN,0.033502
1,bucket,I,bucket,NaN,0.018969
2,repair,S,repair,NaN,0.017499
3,grease,I,grease,NaN,0.017377
4,leak,P,leak,NaN,0.016591


In [5]:
# merge and cleanse NLP-containing columns of the data
nlp_select = kex.NLPSelect(columns=['OriginalShorttext'])
raw_text = nlp_select.transform(df)  # Series
raw_text

0                             jump start
1                             jump start
2                             jump start
3                             jump start
4                             jump start
                      ...               
5480       rf   replace r h a c vee belt
5481       lh air cond compressor seized
5482    rf   replace driers on front a c
5483              rf   change compressor
5484    replace water pump   thermostats
Name: OriginalShorttext, Length: 5485, dtype: object

In [7]:
# TODO: make sure tokens are aliased
tex = kex.TokenExtractor()
# toks = tex.fit_transform(raw_text)
replaced_text = kex.token_to_alias(raw_text, vocab)  # raw_text, with token-->alias replacement
toks = tex.fit_transform(replaced_text)

TypeError: object of type 'int' has no len()

In [6]:
bio = kex.iob_extractor(raw_text, vocab)
bio

KeyboardInterrupt: 